In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../../Functions and Dictionaries/") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [19]:
conn = sq.connect('../../../../Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_Annual_2015to2020]', conn)
data = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson' 'All']
theotherlist = ['Tennessee', 'United states']
thefinallist = ['2019', '2020']
data = data.loc[(data['State Name'].isin(theotherlist))]
# data = data.loc[(data['County'].isin(thelist))&
#                 (data['NAICS Description'] == 'Total')&(data['Enterprise Size'] == '2: <20 employees')&
#                 (data['Year'].isin(thefinallist))
#                ]

In [21]:
data.tail()

,StateFIPS,State Name,CountyFIPS,County,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID
234006,47,Tennessee,All,All,99,Industries not classified,01: Total,142,142,241,4733,2017,0500000US47All
234007,47,Tennessee,All,All,99,Industries not classified,02: <5 employees,133,133,141,3036,2017,0500000US47All
234008,47,Tennessee,All,All,99,Industries not classified,03: 5-9 employees,6,6,36,1082,2017,0500000US47All
234009,47,Tennessee,All,All,99,Industries not classified,06: <20 employees,140,140,186,4223,2017,0500000US47All
234010,47,Tennessee,All,All,99,Industries not classified,19: <500 employees,142,142,241,4733,2017,0500000US47All


In [52]:
conn = sq.connect('../../../../Data-Pipelines/Outputs/Labor_Economy.db')
sql_query = pd.read_sql('SELECT * FROM [SUSB_CountyAnnual_2015to2020]', conn)
df = pd.DataFrame(sql_query)
thelist = ['Cheatham', 'Davidson', 'Dickson', 'Houston', 'Humphreys', 'Montgomery', 'Robertson', 'Rutherford', 
          'Stewart', 'Sumner', 'Trousdale', 'Williamson', 'Wilson']
theotherlist = ['Tennessee', 'United States']
data = data.loc[(data['State Name'].isin(theotherlist))&(df['County Name'].isin(thelist))&
                (df['NAICS Description'] == 'Total')&(df['Enterprise Size'] == '1: Total')]

In [53]:
data.head(2)

,StateFIPS,State Name,CountyFIPS,County Name,NAICS,NAICS Description,Enterprise Size,Firms,Establishments,Employment,Annual Payroll ($1000),Year,GEO_ID,County Option Two
391886,47,Tennessee,021,Cheatham,--,Total,2: <20 employees,469,470,1804,67318,2020,0500000US47021,"Cheatham, Tennessee"
392884,47,Tennessee,037,Davidson,--,Total,2: <20 employees,11826,11905,51891,2539427,2020,0500000US47037,"Davidson, Tennessee"


In [54]:
data = data[['County Name', 'Establishments', 'Year']].reset_index(drop = True)
data = data.rename(columns = {'Establishments':'Small Business Establishments'})
df = df[['County Name', 'Establishments', 'Year']].reset_index(drop = True)
df = df.rename(columns = {'Establishments':'Total Establishments'})

In [55]:
data = data.merge(df, on = ['County Name', 'Year'])
data.head()

,County Name,Small Business Establishments,Year,Total Establishments
0,Cheatham,470,2020,637
1,Davidson,11905,2020,20512
2,Dickson,658,2020,1017
3,Houston,72,2020,100
4,Humphreys,224,2020,329


In [56]:
cols = ['Small Business Establishments', 'Total Establishments']
data[cols] = data[cols].astype(float)

In [57]:
data['% of Establishments Small Businesses'] = data['Small Business Establishments']/data['Total Establishments']
thelist = ['2019', '2020']
data = data.loc[data['Year'].isin(thelist)]

In [58]:
data

,County Name,Small Business Establishments,Year,Total Establishments,% of Establishments Small Businesses
0,Cheatham,470.0,2020,637.0,0.737834
1,Davidson,11905.0,2020,20512.0,0.580392
2,Dickson,658.0,2020,1017.0,0.647001
3,Houston,72.0,2020,100.0,0.720000
4,Humphreys,224.0,2020,329.0,0.680851
5,Montgomery,1870.0,2020,3090.0,0.605178
6,Robertson,804.0,2020,1221.0,0.658477
7,Rutherford,3313.0,2020,5631.0,0.588350
8,Stewart,113.0,2020,164.0,0.689024
9,Sumner,2224.0,2020,3391.0,0.655854


In [59]:
data.to_csv('../data/susb.csv', index = False)